In [4]:
using Pkg; for p in ("Embeddings","DataStructures","DataFrames","FileIO","LinearAlgebra","Knet","FileIO"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using DataStructures,DataFrames,FileIO,Embeddings,LinearAlgebra
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data

In [2]:
#read data from saved file 
revs, W, W2, word_idx_map, vocab=load("TREC_sib.jld2","datas");
word_idx_map["ROOT"] = 0

5-element Array{Any,1}:
 Any[Dict{String,Any}("y"=>2,"num_words"=>10,"tree"=>Any[Any["ROOT", "ROOT", "ROOT", "did", "develop", "did", "develop", "How", "*START*", "ROOT"], Any["ROOT", "ROOT", "did", "develop", "ROOT", "then", "leave", "*START*", "*START*", "develop"], Any["ROOT", "did", "develop", "ROOT", "ROOT", "Russia", "leave", "*STOP*", "*STOP*", "develop"], Any["did", "develop", "ROOT", "ROOT", "ROOT", "?", "develop", "*STOP*", "*STOP*", "ROOT"], Any["did", "develop", "ROOT", "ROOT", "ROOT", "leave", "develop", "?", "*STOP*", "ROOT"], Any["then", "leave", "develop", "ROOT", "ROOT", "and", "develop", "leave", "?", "ROOT"], Any["Russia", "leave", "develop", "ROOT", "ROOT", "develop", "ROOT", "*STOP*", "*STOP*", "ROOT"], Any["?", "develop", "ROOT", "ROOT", "ROOT", "serfdom", "develop", "How", "did", "ROOT"], Any["leave", "develop", "ROOT", "ROOT", "ROOT", "in", "develop", "and", "leave", "ROOT"], Any["and", "develop", "ROOT", "ROOT", "ROOT", "How", "develop", "*START*", "*START*", "

In [254]:
#how the source code is representing the question text as matrix 
function get_text_mat(t,word_idx_map,filter_h=5)
    #t the text of question
    x=[] # output matrix
    pad=filter_h -1 # padding number
    for i in collect(1:pad);push!(x,0);end #adding padding 
    words=split(t)
    #extract the unique id of words in the question text and adding it to the matrix 
    for w in words
        if w in keys(word_idx_map);push!(x,word_idx_map[w])
        else; @show w ;end
    end    

    while length(x)<56+2*pad    # accomplish 64 +1 size by adding zeros till finish 
            push!(x,0)
    end
    
    return  x
end

get_text_mat (generic function with 1 method)

In [255]:
#how the source code is representing the question tree as matrix 
function get_tree_rep(r,word_idx_map)
# question 
#@show t=r["tree"] #the tree of question
    each_sent=deepcopy(t)# output matrix
    for (j, each_word) in enumerate(each_sent[1:end-1])
        #@show (j, each_word)
            for (l, each_field) in enumerate(each_word)
           # @show (l, each_field)
                if each_field in keys( word_idx_map)
                #@show j,l ;
                    each_sent[j]=Array{Any,1}(each_sent[j])
                     each_sent[j][l] = word_idx_map[each_field]
                elseif each_field == 0
                    continue
                else
                    @show each_field
                end
            end
    end       
    return each_sent;
end

get_tree_rep (generic function with 2 methods)

In [262]:
function train_dev_test(revs)
    s1,s2,s3=[],[],[]
    t1,t2,t3=[],[],[]
    for rev in revs
    sent =get_text_mat(rev["text"], word_idx_map,5)   
    push!(sent,rev["y"])
    sent_tensor = get_tree_rep(rev["tree"], word_idx_map)
        
    if rev["split"]==1
            push!(s1,KnetArray{Int}(sent))
            push!(t1,sent_tensor)
    elseif rev["split"]==2
            push!(s2,KnetArray{Int}(sent))
            push!(t2,sent_tensor)
    elseif rev["split"]==3
            push!(s3,KnetArray{Int}(sent))
            push!(t3,sent_tensor)
    end
end

    train = hcat([f1 for f1 in s1]...)
    test =hcat([f1 for f1 in s2]...)
    dev = hcat([f1 for f1 in s3]...)
    train_tensor = t1
    test_tensor = t2
    dev_tensor = t3
    return (train,test,dev),(train_tensor,test_tensor,dev_tensor)
end
dataset,datasetTensor=train_dev_test(revs);


dataset[1] --> train 65×5451 } text data
dataset[2]----> test 65×501  }
dataset[3] ----> dev 0x0     }

datasetTensor[1] ---> train_tensor 5451(46(10)))
datasetTensor[2] ---> test_tensor 501(46(10)))
datasetTensor[3] --->dev_tensor 0(0(0))

In [ ]:
BATCHSIZE=170
function seqbatch(x,y,B,T)
    N = length(x) ÷ B
    x = permutedims(reshape(x[1:N*B],N,B))
    y = permutedims(reshape(y[1:N*B],N,B))
    d = []; for i in 0:T:N-T
        push!(d, (x[:,i+1:i+T], y[:,i+1:i+T]))
    end
    return d
end
allw = vcat((x->x[1]).(data)...)
allt = vcat((x->x[2]).(data)...)
d = seqbatch(allw, allt, BATCHSIZE, SEQLENGTH);

In [5]:
# Define a convolutional layer:
struct Conv; w; b; f; p; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [6]:
# Redefine dense layer (See mlp.ipynb):
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [7]:
# Let's define a chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)